<a href="https://colab.research.google.com/github/Marcel99Codes/MLSec_project2_MM/blob/main/lab_secml_improoved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip install git+https://github.com/RobustBench/robustbench.git
%pip install secml-torch[foolbox,tensorboard]

import torchvision.datasets
from robustbench.utils import load_model
from secmlt.adv.evasion.perturbation_models import LpPerturbationModels
from secmlt.adv.evasion.pgd import PGD
from secmlt.metrics.classification import Accuracy
from secmlt.models.pytorch.base_pytorch_nn import BasePytorchClassifier
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import torch.nn.functional as F
from secmlt.trackers import (
    GradientNormTracker,
    GradientsTracker,
    LossTracker,
    PerturbationNormTracker,
    PredictionTracker,
    SampleTracker,
    ScoresTracker,
    TensorboardTracker,
)

  Cloning https://github.com/RobustBench/robustbench.git to c:\users\moritz\appdata\local\temp\pip-req-build-y13oh0s8
  Resolved https://github.com/RobustBench/robustbench.git to commit 46a91f44524133b2cd8f721ec7e73ecb63f17fc8
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached autoattack-0.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/RobustBench/robustbench.git 'C:\Users\Moritz\AppData\Local\Temp\pip-req-build-y13oh0s8'


Note: you may need to restart the kernel to use updated packages.


In [12]:
class ComposedClf(nn.Module):
    def __init__(self, models):
        super(ComposedClf,self).__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x):
        """randomly selects a model from the list and returns its prediction. Same results as when using an average
        of the models in terms of the robust accuracy, but faster and more memory efficient"""
        model=self.models[torch.randint(0,len(self.models),(1,)).item()]
        return model(x)

In [13]:
model_names = [
    "Sehwag2021Proxy_R18",
    "Rebuffi2021Fixing_70_16_cutmix_extra",
    "Rade2021Helper_extra"
]

In [14]:
models = [load_model(model_name=model_name) for model_name in model_names]
net = ComposedClf(models)#load_model(model_name="Rony2019Decoupling", dataset="cifar10", threat_model="L2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    net.to(device)

test_dataset = torchvision.datasets.CIFAR10(
    transform=torchvision.transforms.ToTensor(),
    train=False,
    root=".",
    download=True,
)

In [15]:
test_dataset = Subset(test_dataset, list(range(100)))
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [16]:
# Wrap model
model = BasePytorchClassifier(net)

# Test accuracy on original data
accuracy = Accuracy()(model, test_data_loader)
print("Accuracy:", accuracy.item())

Accuracy: 0.9300000071525574


In [17]:
# Create and run attack
epsilon = 8/255
num_steps = 100
step_size = 0.005
perturbation_model = LpPerturbationModels.LINF
y_target = None

trackers = [
    LossTracker(),
    PredictionTracker(),
    PerturbationNormTracker("linf"),
    GradientNormTracker(),
    SampleTracker(),
    ScoresTracker(),
    GradientsTracker(),
]
tensorboard_tracker = TensorboardTracker("example_data/logs/pgd", trackers)

native_attack = PGD(
    perturbation_model=perturbation_model,
    epsilon=epsilon,
    num_steps=num_steps,
    step_size=step_size,
    random_start=False,
    y_target=y_target,
    trackers=tensorboard_tracker,
    loss_function="dlr" #better results than using crossentropy
)
native_adv_ds = native_attack(model, test_data_loader)
print(len(native_adv_ds))

c:\Users\Moritz\miniconda3\envs\mlsec\lib\site-packages\foolbox\models\pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(
c:\Users\Moritz\miniconda3\envs\mlsec\lib\site-packages\foolbox\models\pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(
c:\Users\Moritz\miniconda3\envs\mlsec\lib\site-packages\foolbox\models\pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(
c:\Users\Moritz\miniconda3\envs\mlsec\lib\site-packages\foolbox\models\pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Ca

100


In [1]:
# tensorboard
%load_ext tensorboard
%tensorboard --logdir example_data/logs/pgd

Reusing TensorBoard on port 6006 (pid 2708), started 1 day, 9:44:11 ago. (Use '!kill 2708' to kill it.)

In [19]:
# prompt: find all images in native_adv_ds for which all three models fail to predict the right label and put them into a new dataloader

import torch
from torch.utils.data import DataLoader, Subset

# Assuming native_adv_ds is already defined from the previous code
failed_indices = []
print(len(native_adv_ds))
for i,(x,y) in enumerate(native_adv_ds):
    # Check predictions of all three models
    print(f"Image {i}")
    all_models_failed = True
    for model in models:
        print(model.__class__.__name__)
        with torch.no_grad():
            prediction = model(x).argmax()
            print("pred:", prediction)
            print("true:", y)
        if prediction == y:
            all_models_failed = False
    if all_models_failed:
        failed_indices.append(i)

failed_dataset = Subset(native_adv_ds, failed_indices)
failed_data_loader = DataLoader(failed_dataset, batch_size=5, shuffle=False)

print(f"Number of images where all three models failed: {len(failed_indices)}")


100
Image 0
ResNet
pred: tensor(3)
true: tensor([3])
DMWideResNet
pred: tensor(3)
true: tensor([3])
DMWideResNet
pred: tensor(3)
true: tensor([3])
Image 1
ResNet
pred: tensor(8)
true: tensor([8])
DMWideResNet
pred: tensor(8)
true: tensor([8])
DMWideResNet
pred: tensor(8)
true: tensor([8])
Image 2
ResNet
pred: tensor(1)
true: tensor([8])
DMWideResNet
pred: tensor(8)
true: tensor([8])
DMWideResNet
pred: tensor(1)
true: tensor([8])
Image 3
ResNet
pred: tensor(0)
true: tensor([0])
DMWideResNet
pred: tensor(0)
true: tensor([0])
DMWideResNet
pred: tensor(0)
true: tensor([0])
Image 4
ResNet
pred: tensor(6)
true: tensor([6])
DMWideResNet
pred: tensor(4)
true: tensor([6])
DMWideResNet
pred: tensor(4)
true: tensor([6])
Image 5
ResNet
pred: tensor(6)
true: tensor([6])
DMWideResNet
pred: tensor(6)
true: tensor([6])
DMWideResNet
pred: tensor(6)
true: tensor([6])
Image 6
ResNet
pred: tensor(3)
true: tensor([1])
DMWideResNet
pred: tensor(1)
true: tensor([1])
DMWideResNet
pred: tensor(1)
true: tensor(

In [20]:
# Test accuracy on the composed model
n_robust_accuracy = Accuracy()(BasePytorchClassifier(net), native_adv_ds)
print("Robust Accuracy (PGD Native): ", n_robust_accuracy.item())

# Test accuracy on the single models
for m,name in zip(models, model_names):
  n_robust_accuracy = Accuracy()(BasePytorchClassifier(m), native_adv_ds)
  print(f"Robust Accuracy (PGD Native, {name}) ", n_robust_accuracy.item())





Robust Accuracy (PGD Native):  0.6200000047683716
Robust Accuracy (PGD Native, Sehwag2021Proxy_R18)  0.6299999952316284
Robust Accuracy (PGD Native, Rebuffi2021Fixing_70_16_cutmix_extra)  0.6899999976158142
Robust Accuracy (PGD Native, Rade2021Helper_extra)  0.6800000071525574


In [25]:
# save all created adversarial examples, also the unsuccessful ones
import os
import numpy as np
from PIL import Image

output_dir = "adv_images"
os.makedirs(output_dir, exist_ok=True)

# Create subdirectories for each class
for i in range(10):  # Assuming 10 classes for CIFAR-10
    os.makedirs(os.path.join(output_dir, str(i)), exist_ok=True)

for i, (x, y) in enumerate(native_adv_ds):
    x = x.squeeze().detach().cpu().numpy()
    x = np.moveaxis(x, 0, -1)
    x = (x * 255).astype(np.uint8)
    x = Image.fromarray(x)
    x.save(os.path.join(output_dir, str(y.item()), f"{i}.png"))

#create a dataset using the native_adv_ds
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import ImageFolder

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

adv_dataset = ImageFolder("adv_images", transform=transform)
adv_data_loader = DataLoader(adv_dataset, batch_size=1, shuffle=False)

# pickle the dataset and the dataloader
import pickle

with open("adv_dataset.pkl", "wb") as f:
    pickle.dump(adv_dataset, f)

with open("adv_data_loader.pkl", "wb") as f:
    pickle.dump(adv_data_loader, f)






In [26]:
#test the pickled dataset and dataloader
import pickle
from torch.utils.data import DataLoader

with open("adv_dataset.pkl", "rb") as f:
    adv_dataset = pickle.load(f)

with open("adv_data_loader.pkl", "rb") as f:

    adv_data_loader = pickle.load(f)

# Test accuracy on the composed model
n_robust_accuracy = Accuracy()(BasePytorchClassifier(net), adv_data_loader)
print("Robust Accuracy (PGD Native): ", n_robust_accuracy.item())


Robust Accuracy (PGD Native):  0.6899999976158142
